# Klasy

In [23]:
class Gadget:
    def __init__(self, id: int, name: str) -> None:
        self.id = id
        self.name = name
    
    def use(self) -> None:
        print(f"Using {self.name} with Id#{self.id}")    
        self.use_count = 1

    def __repr__(self) -> str:
        return repr(self.__dict__)
        
    
    def __str__(self) -> str:
        return f"Gadget({self.id}, {self.name})"

In [24]:
my_gadget = Gadget(1, "ipad")
my_gadget.use()

Using ipad with Id#1


In [25]:
my_gadget

{'id': 1, 'name': 'ipad', 'use_count': 1}

In [19]:
eval(repr(my_gadget))

Gadget(id=1, name='ipad')

In [20]:
print(my_gadget)

Gadget(1, ipad)


In [22]:
my_gadget

Gadget(id=1, name='ipad')

## Metody `__new__` i `__init__`

In [26]:
class Dummy:
    def __new__(cls, *args):
        print(f"Dummy.__new__({cls}, {args}) has been called...")
        obj = super().__new__(cls)
        obj.extra_attribute = "extra"
        print(f"Object {obj} has been created...")
        return obj
    
    def __init__(self, *args):
        print(f"Dummy.__init__({self}, {args})...")
        self.args = args
        print(f"Object's __dict__: {self.__dict__}")


In [27]:
d = Dummy(1, "two")

Dummy.__new__(<class '__main__.Dummy'>, (1, 'two')) has been called...
Object <__main__.Dummy object at 0x0000024297785850> has been created...
Dummy.__init__(<__main__.Dummy object at 0x0000024297785850>, (1, 'two'))...
Object's __dict__: {'extra_attribute': 'extra', 'args': (1, 'two')}


## Kiedy używamy `__new__`?

In [28]:
class UppercaseTuple(tuple):
    def __init__(self, list) -> None:
        print(f"Start changes for {list}")
        for i, item in enumerate(list):
            self[i] = item.upper()


In [29]:
UppercaseTuple(["hello", "world"])

Start changes for ['hello', 'world']


TypeError: 'UppercaseTuple' object does not support item assignment

In [46]:
class UppercaseTuple(tuple):
    def __new__(cls, list):
        print(f"Start changes for {list}")
        new_content = [item.upper() for item in list]       
        return super().__new__(cls, new_content)

In [47]:
ut = UppercaseTuple(["hello", "world"])

Start changes for ['hello', 'world']


In [48]:
ut

('HELLO', 'WORLD')